In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### import data


In [ ]:
orders = pd.read_json("../raw_data/order_scoring.json",lines=True)

In [ ]:
orders

### determining how many sku to take into consideration

In [2]:
print(len(orders['sku'].unique())," unique sku")

NameError: name 'orders' is not defined

In [ ]:
print("as we see bellow, we can take only the 1000 more orderd skus")
counted_sku_df = orders.groupby('sku').count()['LO'].copy()
counted_sku_df.sort_values(ascending=False,inplace=True)
sns.histplot(counted_sku_df)
plt.xlim(0,6000)


In [ ]:
sku_lenght = 500

### drop unfamous skus




In [ ]:
short_sku_list = list(counted_sku_df.head(sku_lenght).index)

In [ ]:
orders_short = orders[orders['sku'].isin(short_sku_list)]

# Scoring

## Score each order

In [ ]:
coef = {
    1:0.4,
    2:0.3,
    3:0.2,
    4:0.1
    }

In [ ]:
orders_short['score'] = orders_short['LO'].map(coef)

In [ ]:
table_scoring = orders_short.groupby(by=['customer_id','sku']).sum()[['score']]

table_scoring


## Score penality

In [ ]:
penality = 0.5

In [ ]:
table_penality = pd.pivot_table(orders_short, 
                       values=['score'], 
                       index=['customer_id','sku'],
                       columns=['LO'],
                       aggfunc='count',
                       fill_value=0)

table_penality['penality'] = table[[('score',4),('score',3)]].max(axis=1)*(1-table[('score',2)])*(1-table[('score',1)])*penality
table_penality

## add those two scores

In [ ]:
final_table = pd.DataFrame(table_scoring['score'] - table_penality['penality'],columns=["score"])

In [ ]:
data_df = final_table.reset_index()

In [ ]:
data_df

### trasnpose to double entrey matrix

In [ ]:
final_table

In [ ]:
matrix = pd.pivot_table(final_table, values=['score'], index=['customer_id'],
                    columns=['sku'],aggfunc='sum')


# SVD

In [ ]:
all_skus = orders['sku'].unique()
all_skus
len(all_skus)

### Test scikit-surprise

In [ ]:
# Imports
from surprise import Reader, Dataset, SVD, NMF
from surprise.model_selection import cross_validate, train_test_split
from surprise.accuracy import rmse, mae

Exemple https://towardsdatascience.com/various-implementations-of-collaborative-filtering-100385c6dfe0

Doc scikit-surprise https://surprise.readthedocs.io/en/stable/getting_started.html#load-from-df-example

In [ ]:
# On crée un Reader de scikit-surprise
reader = Reader(rating_scale=(1, 5))

## SVD

In [ ]:
# On crée un modèle SVD
model = SVD()

In [ ]:
data_df

In [ ]:
data = Dataset.load_from_df(data_df, reader)

In [ ]:
# J'essaie de cross-validate, pour voir le score du modèle sans aucune optimisation
cross_validate(model, data, measures=['RMSE'], cv=5, verbose=True)

In [ ]:
# On va faire un training à la main.
# On sépare train et test set
trainset, testset = train_test_split(data, test_size=0.25)

In [ ]:
# On fit le model
model.fit(trainset)

In [ ]:
# On fait des prédictions
predictions = model.test(testset)

In [ ]:
# Résultat
rmse(predictions)

In [ ]:
mae(predictions)

In [ ]:
# On peut faire des prédictions de score, pour un customer et un produit au hasard
# .est -> renvoie le score attendu
model.predict('h6IvomPZxOTKuKGuvS8b3nZrzVk2', '4-LAV-100')

In [ ]:
model.predict('00HCaCqtG9aMGVCDyBUbvzIzY073', '1-FAV-133')

In [ ]:
# Est-ce qu'on peut prédire l'item qui aura la meilleur note ?
# Il faut appeler model.predict() sur tous les items, et choisir les meilleurs notes
customer_id = 'h6IvomPZxOTKuKGuvS8b3nZrzVk2'  # On choisit un customer
scores_for_customer = all_skus.copy()
scores_for_customer

In [ ]:
df_customer = pd.DataFrame(scores_for_customer)